In [1]:
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sqlalchemy import create_engine
from coord_convert.transform import gcj2bd



import pymysql

from tools import *




In [2]:
conn = pymysql.Connect(**MYSQL_MALL_DIC)
cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)
engine_conn = "mysql+pymysql://{user}:{password}@{host}:{port}/{database}".format(**MYSQL_MALL_DIC)
engine = create_engine(engine_conn,encoding='utf-8')

In [3]:
# order_df = pd.read_sql('select * from order_details where 下单时间>="2019-12-01 00:00:00"',engine)
goods_df = pd.read_sql('select * from goods',engine,)
cursor.execute('select * from order_details where 下单时间>="2019-12-01 00:00:00"')
d = cursor.fetchall()
order_df = pd.DataFrame(d)

In [4]:
free_order = order_df[order_df['goods_type']=="free"]

df = pd.merge(free_order,goods_df[["商品ID","SPUID","goods_name",'系统分类','市场价','成本价','售价','发货商','发货商ID','规格','单位']],on='商品ID',how='left')
df['lat'] = df['lat'].astype(np.float64)
df['lng'] = df['lng'].astype(np.float64)

df.reset_index(drop=True,inplace=True)
# 转换为 百度坐标系
def convert_coord(coord):
    coord_r = gcj2bd(*coord.tolist())

    coord['lng'] = coord_r[0]
    coord['lat'] = coord_r[1]
    return coord
df_coord = df[['lng','lat']].apply(convert_coord,axis=1)
del df['lng']
del df['lat']
df = pd.concat([df,df_coord],axis=1)

In [5]:
save_dir = r"D:\往期\QHJ\echart"

In [6]:
pd.set_option('mode.chained_assignment',None)
# 运单未回部分
df_r = df[['goods_name','下单时间','订单号','导出订单时间','运单号']]
df_r = df_r.replace({"goods_name":{r".+?\((.+?)\)$":r"\1"}},regex=True)
df_r['发货状态'] = "已回复"

df_r['发货状态'][df_r['运单号'].isnull()] = "待回复"
del df_r['运单号']

df_r['对接日期'] = df_r['导出订单时间'].apply(lambda x:x.strftime('%Y-%m-%d'))
df_r['下单日期'] = df_r['下单时间'].apply(lambda x:x.strftime('%Y-%m-%d'))
df_r.sort_values('导出订单时间',inplace=True)
df_r.reset_index(drop=True,inplace=True)

# 当天订单 当天对接
df_r['current_day'] = "非当日"
df_r['current_day'][df_r['对接日期'] == df_r['下单日期']] = "当日"
data_d = {}

In [7]:
# 回复状态
temp = df_r[['对接日期','发货状态','订单号']].groupby(['对接日期','发货状态']).count()
temp = temp.unstack(fill_value=0)



temp.columns = temp['订单号'].columns
data_d['reply_status'] = temp



# # 回复状态  小细节未发货部分
temp = df_r[df_r['发货状态'] == '待回复']    


temp = temp[['goods_name','对接日期','订单号']]
temp.rename(columns={"goods_name":"三金商品"},inplace=True)
temp = temp.groupby(['三金商品','对接日期']).count()
temp = temp.unstack(level=0).fillna(0)
temp = temp.stack(level=0).reset_index(level=-1,drop=True)    
data_d['reply_status_day'] = temp


# 当日下单数
temp = df_r[['下单日期','订单号']]
temp.rename(columns={"订单号":"下单数"},inplace=True)
temp = temp.groupby(['下单日期']).count()
data_d['order'] = temp

# 对接日期 和 下单日期 是否 为同一天
temp = df_r[['current_day','订单号','对接日期']]
temp = temp.groupby(['对接日期','current_day']).count()
temp = temp.unstack(fill_value=0).sort_index()
temp.columns = temp['订单号'].columns
data_d['abut_cur_date'] = temp

df =pd.concat([df for df in data_d.values()],axis=1,sort=True).fillna(0).sort_index()



In [8]:
# 回复状态
data = {}
data["xaxis"] = df.index.tolist()
data["abut_data"] = []
recv_col = ['已回复','待回复']
for col in set(recv_col) - set(df.columns):
    recv_col.remove(col)

for name, row in df[recv_col].iteritems():
    d = {
        'name':name,
        'data':row.tolist()
    }
    data["abut_data"].append(d)
    
data['order_data'] = {'name':"下单数",'data':df['下单数'].tolist()}
# 回复状态 细节部分
data["not_abut_data"] = {}
columns = ["护肤","水光针","美肤","补水"]
for col in set(columns) - set(df.columns):
    columns.remove(col)

for name,row in df[columns].iterrows():
    r = row!=0
    if any(r):
        data["not_abut_data"][name] = row[r].to_dict()

# 对接日期 和 下单日期 是否 为同一天        
data["cur_vs_abut_order"] = {}    
for name,row in df[['当日','非当日']].iterrows():
    data["cur_vs_abut_order"][name] = row.to_dict()        


# 订单细节部分
temp = df_r[['goods_name','下单日期','订单号']]
temp.rename(columns={'订单号':"下单数"},inplace=True)
temp = temp.groupby(['下单日期','goods_name']).count()
temp = temp.unstack(fill_value=0)
temp.columns = temp['下单数'].columns
temp = temp.sort_index()
data["order_details"] = {}
for name,row in temp.iterrows():
    r = row!=0
    if any(r):
        data["order_details"][name] = row[r].to_dict()
        

# 占比
temp = df_r[['goods_name','订单号']]

temp = temp.rename(columns= {'订单号':"下单数"}).groupby('goods_name').count()

data["goods_order"] = []
for name, row in temp.iterrows():
    d = {
        "name":name,
        "value":row['下单数']
    }
    data["goods_order"].append(d)

        
        
js_path = os.path.join(save_dir,r'data\order.json')
with open(js_path,"w",encoding='utf-8') as f:

    f.write('order_data=')

    json.dump(data,f,ensure_ascii=False,cls=MyEncoder)    

In [9]:

stop


        

NameError: name 'stop' is not defined

In [ ]:
df_r.columns

In [ ]:
temp = df_r[['goods_name','下单日期','订单号']]
temp.rename(columns={'订单号':"下单数"},inplace=True)
temp = temp.groupby(['下单日期','goods_name']).count()
temp = temp.unstack(fill_value=0)
# temp.columns = temp['下单数'].columns
# temp = temp[columns].sort_index()
temp

In [ ]:

temp

In [ ]:
row

In [ ]:
from pprint import pprint
pprint(data)

In [ ]:
# 柱状图
df_r = df[['goods_name','下单时间','订单号']]
df_r = df_r.replace({"goods_name":{r".+?\((.+?)\)$":r"\1"}},regex=True)
df_r['下单日期'] = df_r['下单时间'].apply(lambda x:x.strftime('%Y-%m-%d'))
df_r.sort_values(by=['goods_name','下单时间'],inplace=True)

df_r = df_r[['goods_name','下单日期','订单号']]
df_r.rename(columns={"goods_name":"三金商品"},inplace=True)
df_r = df_r.groupby(['三金商品','下单日期']).count()
df_r = df_r.unstack(level=0).fillna(0)
df_r = df_r.stack(level=0).reset_index(level=-1,drop=True)

data = {}

data["order_data"] = []
data["xaxis"] = df_r.index.tolist()
for name, row in df_r[["补水","美肤","护肤","水光针"]].iteritems():
    d = {
        'name':name,
        'data':row.tolist()
    }
    data["data"].append(d)

js_path = os.path.join(save_dir,r'data\order.json')
with open(js_path,"w",encoding='utf-8') as f:

    f.write('order_data=')

    json.dump(data,f,ensure_ascii=False)
df_r

In [ ]:
# 热图
df_r = df[['订单号','lng','lat']]
df_r = df_r.replace({"goods_name":{r".+?\((.+?)\)$":r"\1"}},regex=True)
l = []
for index, row in df_r.iterrows():
    l.append([row['lng'],row['lat'],1])

    
with open(os.path.join(save_dir,r'\data\heat_coord.json'),'w',encoding='utf-8') as f:
    f.write('points=')
    json.dump(l,f,ensure_ascii=False)

In [ ]:
# map  散点图

temp = df[[ "goods_name",'收货地址','lng','lat','订单号','收件人','联系方式']]
temp = temp.replace({"goods_name":{r".+?\((.+?)\)$":r"\1"}},regex=True)



l = []
for index,row in temp.iterrows():
    d = {
        "name":row['收货地址'],
    }
    lng, lat = gcj2bd(float(row['lng']),float(row['lat']))
    d['value'] = [lng, lat, 1,{"商品名":row['goods_name'],
                                             "收货地址":row['收货地址'],
                                             "收件人":row['收件人'],
                                            "联系方式":row['联系方式']}]
    l.append(d)

with open(os.path.join(save_dir,r'data\coord.json'),'w',encoding='utf-8') as f:
    f.write('coord_data=')
    json.dump(l,f,ensure_ascii=False)

In [ ]:
# 统计
df_r = df[['goods_name','订单号','国家','省份','城市','县区']]
df_r = df_r.replace({"goods_name":{r".+?\((.+?)\)$":r"\1"}},regex=True)

dt = pd.pivot_table(df_r,
         index=['国家','省份','城市','县区'],
         columns=['goods_name'],
         values=['订单号'],
         aggfunc={'订单号':"count"} ,fill_value=0,
               margins=True,
               margins_name="总计"
        )
dt = dt.stack(level=0).reset_index(level=-1,drop=True)
dt.columns.name = "三金护肤产品"
dt = dt[['水光针','补水','美肤','护肤','总计']].sort_index(level=1,ascending=False)
dt

In [ ]:
stop



In [ ]:
plt.rcParams['font.sans-serif'] = ['KaiTi'] # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False
df_r[['补水','护肤','护肤','水光针']].plot(kind='line',figsize=(16,6),rot = 45,
       grid = True)

In [ ]:
free_order.to_excel(r"C:\Users\qhj01\Desktop\TTTT.xlsx")